In [1]:
import numpy as np
import pandas as pd

# Grouped ICA based Time series data

In [2]:
# load ICA based time series data and label
time_series1=np.load("E:/ADHD200/ICA_times_series_905.npy",allow_pickle=True)
label= np.load("E:/ADHD200/label_905.npy")


# Grouped DL based time series data

In [3]:
# load DL based time series data and label
time_series2=np.load("E:/ADHD200/dict_learning_times_series_905.npy",allow_pickle=True)
label= np.load("E:/ADHD200/label_905.npy")


# Concatenate time series data

In [4]:
time_series = []
for i in range(len(time_series1)):
    #concatenate the time series data of grouped ICA and DL
   time_series.append(np.concatenate((time_series1[i], time_series2[i]), axis=1))

In [5]:
np.array(time_series[0]).shape

(152, 93)

# Time domain feature set generation

In [159]:
from scipy.stats import skew
from scipy.stats import kurtosis
def entropy_cal(data):
    data = np.array(data)
    
    # Calculate the probability distribution
    unique_elements, counts = np.unique(data, return_counts=True)
    probabilities = counts / len(data)
    
    # Calculate the entropy
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

def calculate_skewness(data):
    # Convert data to a numpy array if it's not already
    data = np.array(data)
    
    # Calculate skewness
    skewness_value = skew(data)
    
    return skewness_value

def calculate_kurtosis(data):
    # Convert data to a numpy array if it's not already
    data = np.array(data)
    
    # Calculate mean, standard deviation, and kurtosis
    mean_val = np.mean(data)
    std_val = np.std(data, ddof=1)  # Use ddof=1 for sample standard deviation
    kurtosis_value = np.mean((data - mean_val) ** 4) / std_val ** 4
    
    return kurtosis_value

a=[]
one_roi=[]
all_sub=[]
for i in range(0,len(time_series)):
        for j in range(0,time_series[i].shape[1]):
            a.append(np.mean(time_series[i][:,j]))
            a.append(np.std(time_series[i][:,j]))
            a.append(entropy_cal(time_series[i][:,j]))
            a.append(calculate_skewness(time_series[i][:,j]))
            a.append(calculate_kurtosis(time_series[i][:,j]))
            one_roi.append(a)
            a=[]
        all_sub.append(one_roi)
        one_roi=[]

In [160]:
np.array(all_sub).shape

(941, 93, 5)

In [161]:
# save time domain features at drive
np.save("E:/ADHD200/plv_corr/time_domain_features.npy", all_sub)

# Frequency domain feature set generation

In [183]:
import numpy as np
from scipy.signal import welch
from scipy.stats import entropy

def calculate_power_spectral_density(data):
    # Convert data to a numpy array if it's not already
    data = np.array(data)
    
    # Compute the Power Spectral Density (PSD)
    freq_spectrum = np.fft.fft(data)
    psd = np.abs(freq_spectrum) ** 2
    
    # Calculate corresponding normalized frequency values
    num_samples = len(data)
    frequencies = np.fft.fftfreq(num_samples, d=1/sampling_rate)
    
    return frequencies, psd

def calculate_spectral_entropy(data, fs):
    # Convert data to a numpy array if it's not already
    data = np.array(data)
    
    # Calculate the power spectral density (PSD)
    frequencies, psd = welch(data, fs=fs)
    
    # Normalize the PSD to obtain a probability distribution
    psd_normalized = psd / np.sum(psd)
    
    # Calculate the spectral entropy
    spectral_entropy = entropy(psd_normalized, base=2)
    
    return spectral_entropy

import numpy as np

def calculate_frequency_centroid(time_series_data, fs):
    # Convert data to a numpy array if it's not already
    data = np.array(time_series_data)

    # Compute the power spectral density (PSD) using FFT
    fft_vals = np.fft.fft(data)
    psd = np.abs(fft_vals) ** 2

    # Calculate the frequencies corresponding to FFT components
    n = len(data)
    frequencies = np.fft.fftfreq(n, d=1.0 / fs)

    # Calculate the total power
    total_power = np.trapz(psd, frequencies)

    # Calculate the frequency centroid
    frequency_centroid = np.sum(psd * frequencies) / total_power

    return frequency_centroid

a=[]
one_roi=[]
all_sub=[]
sampling_rate = 2  # Sampling rate (Hz)
for i in range(0,len(time_series)):
        for j in range(0,time_series[i].shape[1]):
            frequencies, psd = calculate_power_spectral_density(time_series[i][:,j])
            a.append(psd)
            spectral_entropy_result = calculate_spectral_entropy(time_series[i][:,j], sampling_rate)
            a.append(spectral_entropy_result)
            ctd=calculate_frequency_centroid(time_series[i][:,j],sampling_rate)
            a.append(ctd)
            one_roi.append(a)
            a=[]
        all_sub.append(one_roi)
        one_roi=[]

In [ ]:
# save frequecy domain features at drive
np.save("E:/ADHD200/plv_corr/frequency_domain_features.npy", all_sub)

In [ ]:
pip install python_speech_features

# Time-Frequency domain data

In [ ]:
import numpy as np
import scipy.stats
import pywt
#import librosa
from scipy.signal import spectrogram

# Calculate all feature values 
def time_frequency_entropy(spectrogram_data):
    return -np.sum(spectrogram_data * np.log2(spectrogram_data), axis=0)

def continuous_wavelet_transform(data):
    return pywt.cwt(data, np.arange(1, 128), 'mexh')

def time_frequency_variance(cwt_matrix):
    return np.var(np.abs(cwt_matrix), axis=0)

def wavelet_packet_transform(data):
    wp = pywt.WaveletPacket(data, wavelet='db1', mode='symmetric', maxlevel=4)
    return np.array([node.data.sum() for node in wp.get_level(4, 'freq')])

def mel_frequency_cepstral_coefficients(data, fs):
    return librosa.feature.mfcc(data, sr=fs, n_mfcc=13)

a=[]
one_roi=[]
all_sub=[]
sampling_rate = 2 
for i in range(0,len(time_series)):
        for j in range(0,time_series[i].shape[1]):
            # Calculate the STFT to obtain the spectrogram
             frequencies, times, spectrogram_data = spectrogram(time_series[i][:,j], fs=sampling_rate)
            # Calculate individual time-frequency domain features
             time_freq_entropy_result = time_frequency_entropy(spectrogram_data)
             cwt_matrix, frequencies_cwt = continuous_wavelet_transform(time_series[i][:,j])
             time_freq_variance_result = time_frequency_variance(cwt_matrix)
             wpt_energy_result = wavelet_packet_transform(time_series[i][:,j])
             #mfccs_result = mel_frequency_cepstral_coefficients(time_series[i][:,j], fs=sampling_rate)
             a.append(time_freq_entropy_result)
             a.append(frequencies_cwt[np.argmax(np.abs(cwt_matrix), axis=0)])
             a.append(time_freq_variance_result)
             a.append(wpt_energy_result)
             #a.append(mfccs_result)
             one_roi.append(a)
             a=[]
        all_sub.append(one_roi)
        one_roi=[]

In [ ]:
np.save("E:/ADHD200/plv_corr/time_frequency_domain_features.npy", all_sub)

# PLV based connectivity matrix  generation

In [13]:
import numpy as np

# defining phase function 
def compute_phase(signal):
    """Compute the phase of the signal using the Hilbert transform."""
    hilbert_transform = np.fft.fft(signal)
    """"gate phas angle."""
    phase = np.angle(hilbert_transform)
    return phase

# calculating plv value
def compute_plv(signal1, signal2):
    """Compute Phase-Locking Value (PLV) between two signals."""
    phase1 = compute_phase(signal1)
    phase2 = compute_phase(signal2)

    # Compute the phase difference
    phase_difference = np.angle(np.exp(1j * (phase1 - phase2)))

    # Compute the PLV
    plv = np.abs(np.mean(np.exp(1j * phase_difference)))

    return plv

# plv based matrix generation
# time_series stores the concatenated time series data 
each_roi=[]
all_roi=[]
all_sub=[]
if __name__ == "__main__":
    for i in range(0,len(time_series)):
        for j in range(0,time_series[i].shape[1]):
            for k in range(0, time_series[i].shape[1]):
                    plv_value = compute_plv(time_series[i][:,j], time_series[i][:,k])
                    each_roi.append(plv_value)
            all_roi.append(each_roi)       
            each_roi=[]
        all_sub.append(all_roi)
        all_roi=[]

In [14]:
# save the data in drive 
np.save("E:/ADHD200/plv_corr/all_plv.npy", all_sub)

In [50]:
# load data
X= np.load("E:/ADHD200/plv_corr/all_plv.npy")
Y= np.load("E:/ADHD200/label_941.npy")

# Threshold calculation and adjacency matrix generation

In [8]:
import numpy as np
# load data
X= np.load("E:/ADHD200/plv_corr/all_plv.npy")
Y= np.load("E:/ADHD200/label_905.npy")

X_mean = np.mean(abs(X), axis=0)
sum_of_values = np.sum(X_mean)

#calculate threshold by averaging the mean value of connectivity matrix
threshold= sum_of_values/(X_mean.shape[0]*X_mean.shape[1])
threshold


x_adhd=[]
x_tc=[]
for i in range(0,len(X)):
    if(Y[i]==1):
        x_adhd.append(X[i])
    else:
        x_tc.append(X[i])


x_mean_adhd = np.mean(x_adhd, axis=0)
x_mean_tc = np.mean(x_tc, axis=0)

weighted_adj_adhd_mean = np.where(x_mean_adhd < threshold, 0, x_mean_adhd)
weighted_adj_tc_mean = np.where(x_mean_tc < threshold, 0, x_mean_tc)

res_adhd= np.count_nonzero(weighted_adj_adhd_mean)-93
res_tc= np.count_nonzero(weighted_adj_tc_mean)-93
print(res_adhd, res_tc)

# mean adjacency matrix of adhd and tc
adj_adhd_mean = np.where(x_adhd[0] < threshold, 0, 1)
adj_tc_mean = np.where(x_tc[0] < threshold, 0, 1)


# Finding adjacency matrix based on average thresholding

In [9]:
# Values above threshold is considered as 1, below is 0 
X = np.where(abs(X) < threshold, 0, 1) 
np.save("E:/ADHD200/plv_corr/all_plv_adjacency.npy", X)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot the binary adjacency matrix
plt.figure(figsize=(8, 5))
plt.imshow(adj_adhd_mean, cmap='binary', interpolation='nearest')
plt.colorbar()
plt.xlabel('ROI index',fontsize=15)
plt.ylabel('ROI index',fontsize=15)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot the binary adjacency matrix
plt.figure(figsize=(8, 5))
plt.imshow(adj_tc_mean, cmap='binary', interpolation='nearest')
plt.colorbar()
plt.xlabel('ROI index',fontsize=15)
plt.ylabel('ROI index',fontsize=15)
plt.show()

# Correlation based functional connectivity

In [73]:
time_series1=np.load("E:/ADHD200/ICA_times_series_905.npy",allow_pickle=True)
label= np.load("E:/ADHD200/label_941.npy")

In [74]:
time_series2=np.load("E:/ADHD200/dict_learning_times_series_905.npy",allow_pickle=True)
label= np.load("E:/ADHD200/label_941.npy")

In [75]:
time_series = []
for i in range(len(time_series1)):
   time_series.append(np.concatenate((time_series1[i], time_series2[i]), axis=1))

In [84]:
each_roi=[]
all_roi=[]
all_sub=[]
for i in range(0,len(time_series)):
        for j in range(0,time_series[i].shape[1]):
            for k in range(0, time_series[i].shape[1]):
                    corr = np.corrcoef(time_series[i][:,j], time_series[i][:,k])
                    r = corr[0, 1]
                    each_roi.append(r)
            all_roi.append(each_roi)       
            each_roi=[]
        all_sub.append(all_roi)
        all_roi=[]

In [86]:
np.save("E:/ADHD200/plv_corr/all_corr.npy", all_sub)

In [18]:
X= np.load("E:/ADHD200/plv_corr/all_corr.npy")
Y=np.load("E:/ADHD200/label_905.npy")

# Calculate threshold and correlation based adjacency matrices

In [22]:
X_mean = np.mean(X, axis=0)
sum_of_values = np.sum(X_mean)
#calculate the threshold value
threshold= sum_of_values/(X_mean.shape[0]*X_mean.shape[1])
X = np.where(X < threshold, 0, 1)
X = np.where(abs(X) < threshold, 0, 1) 
np.save("E:/ADHD200/plv_corr/all_corr_adjacency.npy", X)

In [ ]:
x_adhd=[]
x_tc=[]
for i in range(0,len(X)):
    if(Y[i]==1):
        x_adhd.append(X[i])
    else:
        x_tc.append(X[i])

x_mean_adhd = np.mean(x_adhd, axis=0)
x_mean_tc = np.mean(x_tc, axis=0)

weighted_adj_adhd_mean = np.where(x_mean_adhd < threshold, 0, x_mean_adhd)
weighted_adj_tc_mean = np.where(x_mean_tc < threshold, 0, x_mean_tc)

res_adhd= np.count_nonzero(weighted_adj_adhd_mean)-93
res_tc= np.count_nonzero(weighted_adj_tc_mean)-93
print(res_adhd, res_tc)

In [ ]:
x_adhd=[]
x_tc=[]
for i in range(0,len(X)):
    if(Y[i]==1):
        x_adhd.append(X[i])
    else:
        x_tc.append(X[i])

x_mean_adhd = np.mean(x_adhd, axis=0)
x_mean_tc = np.mean(x_tc, axis=0)

# mean adjacency matrix of adhd and tc
adj_adhd_mean = np.where(x_mean_adhd < threshold, 0, 1)
adj_tc_mean = np.where(x_mean_tc < threshold, 0, 1)

res_adhd= np.count_nonzero(adj_adhd_mean)-93
res_tc= np.count_nonzero(adj_tc_mean)-93
print(res_adhd, res_tc)

# Spread of correlation and plv based connectivity strength

In [ ]:
X1= np.load("E:/ADHD200/plv_corr/all_plv.npy")
X2= np.load("E:/ADHD200/plv_corr/all_corr.npy")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sample PLV and correlation-based connectivity matrices (replace with actual data)
X_plv =  np.mean(X1, axis=0)
X_corr = np.mean(X2, axis=0)

# Flatten the matrices to get connectivity strength values
plv_strength = X_plv.flatten()
corr_strength = X_corr.flatten()

# Plot histograms of connectivity strength for both PLV and correlation-based matrices
plt.figure(figsize=(10, 5))
plt.hist(plv_strength, bins=50, alpha=0.5, color='blue', label='PLV-Based')
plt.hist(corr_strength, bins=50, alpha=0.5, color='green', label='Correlation-Based')
plt.xlabel('Connectivity Strength')
plt.ylabel('Frequency')
plt.title('Distribution of Connectivity Strength')
plt.legend()
plt.show()

# Kullback-Leibler Divergence calculation

In [ ]:
# load the plv connectivity matrix
X= np.load("E:/ADHD200/plv_corr/all_plv.npy")

In [ ]:
# calculate mean connectivity matrix
X_mean = np.mean(X, axis=0)
sum_of_values = np.sum(X_mean)
#calculate the threshold value
threshold= sum_of_values/(X_mean.shape[0]*X_mean.shape[1])
x_adhd=[]
x_tc=[]
for i in range(0,len(X)):
    if(Y[i]==1):
        x_adhd.append(X[i])
    else:
        x_tc.append(X[i])

x_mean_adhd = np.mean(x_adhd, axis=0)
x_mean_tc = np.mean(x_tc, axis=0)


# weighted mean matrix generation
weighted_adj_adhd_mean = np.where(x_mean_adhd < threshold, 0, x_mean_adhd)
weighted_adj_tc_mean = np.where(x_mean_tc < threshold, 0, x_mean_tc)


In [ ]:
import numpy as np
import matplotlib

def kl_divergence(adj_matrix1, adj_matrix2, epsilon=1e-10):
    # Normalize adjacency matrices to obtain probability distributions
    prob_dist1 = (adj_matrix1 + epsilon) / np.sum(adj_matrix1 + epsilon, axis=1, keepdims=True)
    prob_dist2 = (adj_matrix2 + epsilon) / np.sum(adj_matrix2 + epsilon, axis=1, keepdims=True)
    
    # Compute KL divergence
    kl_div = np.sum(prob_dist1 * np.log(prob_dist1 / prob_dist2), axis=1)
    
    return kl_div

# Example usage:
# Assuming adj_matrix1 and adj_matrix2 are the two adjacency matrices to be compared

kl_div = kl_divergence(weighted_adj_adhd_mean, weighted_adj_tc_mean)
print("KL Divergence:", np.mean(kl_div))
print(len(kl_div))
matplotlib.rcParams.update({'font.size': 15})

# Assuming kl_div is your list/array of KL divergences

# Plot KL divergence
plt.scatter(range(len(kl_div)), kl_div, label='KL Divergence')

# Calculate and plot average KL divergence
average_kl_div = np.mean(kl_div)
plt.axhline(y=average_kl_div, color='r', linestyle='--', label='Average KL Divergence')

# Add labels and legend
plt.xlabel('Nodes')
plt.ylabel('KL Divergence')
plt.title('Kullback-Leibler Divergence')
plt.legend()

plt.show()

# HURST Exponent calculation

# ADHD hurst exponent calculation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def hurst_exponent_roi(time_series_roi):
    """Calculate the Hurst exponent of a time series for a single ROI."""
    N = len(time_series_roi)
    
    # Calculate the range of the time series
    range_vals = np.ptp(time_series_roi)
    
    # Calculate the mean of the time series
    mean = np.mean(time_series_roi)
    
    # Calculate the cumulative deviation from the mean
    cum_dev = np.cumsum(time_series_roi - mean)
    
    # Calculate the range of cumulative deviation
    R = np.max(cum_dev) - np.min(cum_dev)
    
    # Calculate the standard deviation of the time series
    S = np.std(time_series_roi)
    
    # Calculate the rescaled range
    R_S = R / S
    
    # Calculate the Hurst exponent
    hurst = np.log(R_S) / np.log(N)
    
    return hurst

# Example time series data with shape (a x b)
time_series_data = np.array(tc_timeseries)  # Example data with 1000 time points and 10 ROIs

# Calculate the Hurst exponent for each ROI
hurst_exponents = []
for roi_index in range(time_series_data[4].shape[1]):
    hurst_roi = hurst_exponent_roi(time_series_data[4][:, roi_index])
    hurst_exponents.append(hurst_roi)

plt.rcParams.update({'font.size': 14, 'legend.fontsize': 12})
plt.figure(figsize=(8, 6))
x_values = range(len(hurst_exponents))
y_values = hurst_exponents

# Plot markers for nonlinear Hurst exponents
nonlinear_indices = [i for i, hurst in enumerate(hurst_exponents) if np.abs(hurst - 0.5) > 0.1]
plt.scatter(np.array(x_values)[nonlinear_indices], np.array(hurst_exponents)[nonlinear_indices], marker='*', color='blue', s=80, label='Nonlinear (Hurst Exponent ≠ 0.5)')

# Plot markers for linear Hurst exponents
linear_indices = [i for i, hurst in enumerate(hurst_exponents) if np.abs(hurst - 0.5) <= 0.1]
plt.scatter(np.array(x_values)[linear_indices], np.array(hurst_exponents)[linear_indices], marker='o', color='blue', s=80, label='Linear (Hurst Exponent ~ 0.5)')

# Draw a horizontal line at y=0.5
plt.axhline(y=0.5, color='red', linestyle='--')

plt.xlabel('ROI Index')
plt.ylabel('Hurst Exponent')
plt.title('Hurst Exponents for Each ROI')
plt.legend()
plt.show()


# TC hurst exponent calculation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def hurst_exponent_roi(time_series_roi):
    """Calculate the Hurst exponent of a time series for a single ROI."""
    N = len(time_series_roi)
    
    # Calculate the range of the time series
    range_vals = np.ptp(time_series_roi)
    
    # Calculate the mean of the time series
    mean = np.mean(time_series_roi)
    
    # Calculate the cumulative deviation from the mean
    cum_dev = np.cumsum(time_series_roi - mean)
    
    # Calculate the range of cumulative deviation
    R = np.max(cum_dev) - np.min(cum_dev)
    
    # Calculate the standard deviation of the time series
    S = np.std(time_series_roi)
    
    # Calculate the rescaled range
    R_S = R / S
    
    # Calculate the Hurst exponent
    hurst = np.log(R_S) / np.log(N)
    
    return hurst

# Example time series data with shape (a x b)
time_series_data = np.array(adhd_timeseries)  # Example data with 1000 time points and 10 ROIs

# Calculate the Hurst exponent for each ROI
hurst_exponents = []
for roi_index in range(time_series_data[10].shape[1]):
    hurst_roi = hurst_exponent_roi(time_series_data[10][:, roi_index])
    hurst_exponents.append(hurst_roi)

plt.rcParams.update({'font.size': 18, 'legend.fontsize': 15})
plt.figure(figsize=(8, 6))
x_values = range(len(hurst_exponents))
y_values = hurst_exponents

# Plot markers for nonlinear Hurst exponents
nonlinear_indices = [i for i, hurst in enumerate(hurst_exponents) if np.abs(hurst - 0.5) > 0.1]
plt.scatter(np.array(x_values)[nonlinear_indices], np.array(hurst_exponents)[nonlinear_indices], marker='*', color='blue', s=80, label='Nonlinear (Hurst Exponent ≠ 0.5)')

# Plot markers for linear Hurst exponents
linear_indices = [i for i, hurst in enumerate(hurst_exponents) if np.abs(hurst - 0.5) <= 0.1]
plt.scatter(np.array(x_values)[linear_indices], np.array(hurst_exponents)[linear_indices], marker='o', color='blue', s=80, label='Linear (Hurst Exponent ~ 0.5)')

# Draw a horizontal line at y=0.5
plt.axhline(y=0.599, color='red', linestyle='--')

plt.xlabel('ROI Index')
plt.ylabel('Hurst Exponent')
plt.title('Hurst Exponents for Each ROI')
plt.legend()
plt.show()
